In [1]:
import chess
import chess.engine
import time
import asyncio
from IPython.display import display, HTML, clear_output

In [2]:
def display_board(board, use_svg):
    """
    Displays the chess board.
    :param board: a python-chess board.
    :param use_svg: boolean if visual html animation of board active.
    :return: str representation of html formatted board for display.
    """

    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

In [3]:
def who(agent):
    """
    Checks if player agent is white or black.
    :param agent: a python-chess player.
    :return: str representation of agent's color.
    """

    return "White" if agent == chess.WHITE else "Black"

In [4]:
def get_move(prompt):
    """
    Obtains available moves.
    :param prompt: keyboard interrupt prompt to halt active game.
    :return: str representation of Universal Chess Interface (UCI) move.
    """

    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

In [5]:
def count_pieces(board):
    """
    Tallies the white and black players pieces.
    :param board: a python-chess board.
    :return: arr[int, int] representation of number of pieces on board where
            arr[0] is white and arr[1] is black.
    """
    num_pieces = [0, 0]

    num_pieces[0] += len(board.pieces(chess.PAWN, chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.BISHOP, chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.KING, chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.QUEEN, chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.KNIGHT, chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.ROOK, chess.WHITE))

    num_pieces[1] += len(board.pieces(chess.PAWN, chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.BISHOP, chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.KING, chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.QUEEN, chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.KNIGHT, chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.ROOK, chess.BLACK))

    return num_pieces
 

In [ ]:
# Modified play_game function takes in a stockfish engine and lets stockfish play against itself, 100 milliseconds 
# per move with svg visualization

def play_game(
              engine,
              uci_start_state=None,
              visual="svg",
              pause=0.001):

    use_svg = (visual == "svg")

    if uci_start_state is None:
        board = chess.Board()
    else:
        board = chess.Board(uci_start_state)
    try:
        while not board.is_game_over():
            
            result = engine.play(board, chess.engine.Limit(time=0.1))
            name = who(board.turn)
            board.push(result.move)
            board_stop = display_board(board, use_svg)
            
            html = "<b>Move %s, Play '%s':</b><br/>%s" % (len(board.move_stack), name, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
        engine.quit()
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (False, msg, board)
    game_has_winner = False
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        game_has_winner = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)

    return (game_has_winner, msg, board)

In [ ]:
# Test UCI/XBoard engine communication using the stockfish executable compiled on Ubuntu 18.04.3 LTS with gcc/g++

engine = chess.engine.SimpleEngine.popen_uci("./stockfish")
res1 = play_game(engine, uci_start_state=None, visual="svg", pause=0.001)

In [ ]:
# Test UCI/XBoard engine communication using the stockfish executable compiled on Ubuntu 18.04.3 LTS with gcc/g++,
# giving the engine a FEN start state 

engine2 = chess.engine.SimpleEngine.popen_uci("./stockfish")
res2 = play_game(engine2, uci_start_state="QN4n1/6r1/3k4/8/b2K4/8/8/8 b - - 0 1", visual="svg", pause=0.001)

In [6]:
# Modified play_game function takes in a stockfish engine and lets stockfish play against itself, 100 milliseconds 
# per move with svg visualization

async def play_game(
              transport,
              engine,
              uci_start_state=None,
              visual="svg",
              pause=0.001):

    use_svg = (visual == "svg")

    if uci_start_state is None:
        board = chess.Board()
    else:
        board = chess.Board(uci_start_state)
    try:
        while not board.is_game_over():
            
            result = await engine.play(board, chess.engine.Limit(time=0.1))
            name = who(board.turn)
            board.push(result.move)
            board_stop = display_board(board, use_svg)
            
            html = "<b>Move %s, Play '%s':</b><br/>%s" % (len(board.move_stack), name, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
        await engine.quit()
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (False, msg, board)
    game_has_winner = False
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        game_has_winner = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)

    return (game_has_winner, msg, board)



In [8]:
# Test UCI/XBoard engine communication using the stockfish executable compiled on Ubuntu 18.04.3 LTS with gcc/g++

# transport, engine3 = await chess.engine.popen_uci("./stockfish")
# asyncio.set_event_loop_policy(chess.engine.EventLoopPolicy())
# # asyncio.run(play_game(transport, engine3, uci_start_state=None, visual="svg", pause=0.001))
# res1 = play_game(engine3, uci_start_state=None, visual="svg", pause=0.001)


RuntimeError: Cannot add child handler, the child watcher does not have a loop attached